In [ ]:
import sys
sys.path.append('/home1/yxiao977/labwork')
import test_func 

In [ ]:
request = {
    'hic_path':'/home1/yxiao977/sc1/train_akita/data/5000res_0.5thres_hic_filter_both_bin.cool',
    'cis_total_path':'/home1/yxiao977/sc1/train_akita/data/5000res_bins_cistotal_ratio.bed'
}

test_func.test_cis_total_filter(request)

In [5]:
### Test need to delete later
import cooler
import numpy as np
clr = cooler.Cooler('/home1/yxiao977/sc1/test_func/test.cool')
# chunks = clr.bins()[:10]
# pixels = clr.pixels().fetch('chr1_pilon:0-500000')

In [ ]:
clr.pixels()[:10].loc[0, 'count'] = 0

In [ ]:
clr.pixels()[:10]

In [4]:
import numpy as np
import pandas as pd
import cooltools
import cooler
import logging
def _bins_cis_total_ratio_filter(clr, thres):
    """
    Filter out bins with low cis-to-total coverage ratio from a Cooler object.

    Parameters
    ----------
    clr : cooler.Cooler
        A Cooler object containing Hi-C contact matrices.
    thres : float
        The threshold cis-to-total coverage ratio below which bins are considered bad.

    Returns
    -------
    numpy.ndarray
        An array of bin indices with low cis-to-total coverage ratio.

    Notes
    -----
    This function uses the `cooltools.coverage` function to compute the cis-to-total
    coverage ratio for each bin, and returns an array of bin indices with a ratio
    below `thres`. The returned array contains the indices of bins with low cis-to-total
    coverage ratio, which can be used to filter out these bins from the input Cooler object.

    """
    coverage = cooltools.coverage(clr)
    cis_total_cov = coverage[0] / coverage[1]
    bins_mask = cis_total_cov <= thres

    return bins_mask

/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:652: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_symmetric(
/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:727: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_asymmetric(x, max_iter=10

In [ ]:
def _write_filtered_pixels_files(bin_table, chunk_pixels, bin_mask, count_output_path):
    """
    Write the counts of interactions between genomic regions to a text file.

    This function takes a cooler object containing the genomic regions, a DataFrame containing the counts of interactions
    between genomic regions, and the path to the output text file. It then writes the counts of interactions to the output
    text file.

    Parameters
    ----------
    clr : cooler.Cooler
        A cooler object containing the genomic regions.
    chunk_pixels : pandas.DataFrame
        A DataFrame containing the counts of interactions between genomic regions.
    count_output_path : str
        The path to the output text file.

    Returns
    -------
    None
    
    """
    bad_bins_index = np.array(range(bin_table.shape[0]))[bin_mask]
    pixels_mask = chunk_pixels['bin1_id'].isin(bad_bins_index)
    chunk_pixels.loc[pixels_mask, 'count'] = 0
    pixels_mask = chunk_pixels['bin2_id'].isin(bad_bins_index)
    chunk_pixels.loc[pixels_mask, 'count'] = 0
    # Then write the counts in text file:
    with open(count_output_path, 'a+') as count_file:
        for i, row in chunk_pixels.iterrows():
            bin1_id, bin2_id, count = row
            chrom1, start1, end1 = list(bin_table.values[bin1_id])[:3]
            chrom2, start2, end2 = list(bin_table.values[bin2_id])[:3]

            count_file.write(f"{chrom1}\t{start1}\t{end1}\t{chrom2}\t{start2}\t{end2}\t{count}\n")

In [ ]:
def filter_pixels_by_masked_bin(clr, thres, chromsize_output_path, count_output_path, chunksize=10_000_000):
    """
    Filter the pixels of a cooler object based on a binary mask of masked bins.

    Parameters
    ----------
    clr : cooler.Cooler
        The cooler object to filter.
    thres : float
        The threshold for the cis-total ratio filter.
    chromsize_output_path : str
        The path to the output file where the chromosome sizes will be written.
    count_output_path : str
        The path to the output file where the filtered pixels will be written.
    chunksize : int, optional
        The size of the chunks to process the pixels in, in number of pixels.

    Returns
    -------
    None
    
    """

    logger.debug('Start to make bin mask...')
    bin_mask = _bins_cis_total_ratio_filter(clr, thres)
    tot_pixels = clr.pixels().shape[0]
    iter_num = tot_pixels // chunksize

    # First write the chromosome sizes:
    logger.debug('Start to create chromsize file...')
    with open(chromsize_output_path, 'a+') as chromsize_file:
        for i, chromsize in enumerate(clr.chromsizes):
            chromsize_file.write(f"{clr.chromnames[i]}\t{chromsize}\n")

    logger.debug('Start to create pixels counts file...')
    bins_table = clr.bins()[:]
    for i in range(iter_num):
        logger.debug(f'Start to process pixels chunk {i}...')
        chunk_pixels = clr.pixels()[chunksize * i : chunksize * (i + 1)]
        
        ### Here we might use multiprocessing to boost the speed, but does order matter###
        _write_filtered_pixels_files(bins_table, chunk_pixels, bin_mask, count_output_path)
    
    # remainder 
    chunk_pixels = clr.pixels()[chunksize * iter_num:]
    _write_filtered_pixels_files(bins_table, chunk_pixels, count_output_path)

In [ ]:
import logging
logger = logging.getLogger('one')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s : %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)


In [8]:
thres = 0.5
chromsize_output_path = '/home1/yxiao977/sc1/test_func/chromsize.txt' 
count_output_path = '/home1/yxiao977/sc1/test_func/counts.txt' 
filter_pixels_by_masked_bin(clr, thres, chromsize_output_path, count_output_path, chunksize=10_000_000)

NameError: name 'filter_pixels_by_masked_bin' is not defined

In [ ]:
bin_index = np.array(range(clr.bins().shape[0]))[bin_mask]

In [ ]:
for i, row in chunk_pixels.iterrows():
            bin1_id, bin2_id, count = row
            print(row)
            break

In [9]:
bin_mask = _bins_cis_total_ratio_filter(clr, 0.5)
chunk_pixels = clr.pixels()[10_000_000 * 0 : 10_000_000 * 1]


/tmp/SLURM_16713091/ipykernel_56327/3407875634.py:31: RuntimeWarning: invalid value encountered in true_divide
  cis_total_cov = coverage[0] / coverage[1]


In [15]:
bad_bins_index = np.array(range(clr.bins().shape[0]))[bin_mask]
pixels_mask = chunk_pixels['bin1_id'].isin(bad_bins_index)
pixels_mask2 = chunk_pixels['bin2_id'].isin(bad_bins_index)


In [25]:
type(clr.pixels())

cooler.core._selectors.RangeSelector1D

In [ ]:
pixels_mask = [bin_mask[i] for i in chunk_pixels['bin1_id'].values]

In [ ]:
bin_table = clr.bins()[:]
bad_bins_index = np.array(range(bin_table.shape[0]))[bin_mask]
pixels_mask = chunk_pixels['bin1_id'].isin(bad_bins_index)

In [ ]:
pixels_mask[0] = True

In [ ]:
chunk_pixels.values[pixels_mask, 2] = 0

In [17]:
import h5py

# Open the source and destination HDF5 files
source_file = h5py.File("/home1/yxiao977/sc1/test_func/test.cool", "w")

In [18]:
source_file.keys()

<KeysViewHDF5 []>

In [ ]:


# Copy the dataset from the source file to the destination file
source_file['pixels']['count'][0] = 0

source_file.close()


In [3]:
import numpy as np
np.array([True, False]) + np.array([False, True])

array([ True,  True])

In [20]:
import cooler
t = cooler.Cooler("/home1/yxiao977/sc1/test_func/test.cool")


KeyError: 'No cooler found at: /home1/yxiao977/sc1/test_func/test.cool.'

In [ ]:
len(chunk_pixels['bin1_id'].unique())

In [ ]:
for i in chunk_pixels['bin1_id'].values:
    if bin_mask[i] == False:
        print(i)
        break

In [ ]:
for i, value in enumerate(pixels_mask2):
    if value == True:
        print(i)
        break

In [ ]:
chunk_pixels.iloc[0]

In [ ]:
np.sum(bin_mask)

In [ ]:
pixels_mask2 = chunk_pixels['bin1_id'].isin(bin_mask)

In [ ]:
masked_bin_table = clr.bins()[:]


In [ ]:
masked_bin_table.loc[test_arry,'weight'] = 2

In [ ]:
for i in  pixels['bin1_id'].values:
    print(i)

In [ ]:
test_arry = np.argwhere((np.array([0, 1, 0, 1]) <= 0.5)).reshape(-1)

In [ ]:
import numpy as np

# Sample data
data = np.array([1, 2, 3, 4, 5, 6])

# Create a mask to select even numbers
mask = data % 2 == 0

# Use the mask to filter the data
filtered_data = data[mask]

print(filtered_data)  # Output: [2 4 6]
